# Stroke Prediction

Berdasarkan informasi WHO, stroke merupakan penyebab kematian terbanyak nomor 2 di dunia dan menjadi penyebab dari 11% total kematian. [Dataset](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset) ini digunakan untuk memprediksi apakah seorang pasien memiliki kemungkinan besar untuk terkena stroke berdasarkan informasi tentang pasien seperti jenis kelamin, umur, penyakit dan status merokok.

Tujuan eksperimen:
1.   Peserta memahami rangkaian proses analitika data menggunakan pendekatan pembelajaran mesin. 
2.   Peserta memahami bahwa proses pengembangan model pembelajaran mesin juga ditentukan dari kualitas data, penanganan data, dan penentuan algoritma serta hiperparameternya; tidak cukup hanya dengan memastikan implementasi algoritma berjalan tanpa kesalahan.
3. Peserta mampu menginterpretasikan hasil dari evaluasi model dalam proses analitika menggunakan pendekatan pembelajaran mesin.

Praktikum dilaksanakan secara berkelompok, dengan 1 kelompok terdiri atas 2 mahasiswa. Soal praktikum terdapat di bagian bawah berkas ini. Harap diperhatikan bahwa terdapat berkas yang harus dikumpulkan sebelum waktu praktikum selesai (4 April 2022 11.00 WIB) dan berkas yang dikumpulkan setelah waktu praktikum selesai (4 April 2022 23.59 WIB). Untuk detil deliverables dan soal, dapat dilihat pada bagian bawah notebook

# Data Preparation

In [46]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

In [47]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
X = data.drop(columns="stroke")
y = data["stroke"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [48]:
data.head()
# data['Residence_type'].unique()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# Soal Eksperimen

Disediakan data yang sudah dibagi menjadi data train (df_train), validasi (df_val), dan test (df_test). Lakukanlah:
1. Buatlah baseline dengan menggunakan model Logistic Regression
2. Lakukan analisa data terkait:
- Duplicate value
- Missing value
- Outlier
- Balance of data
3. Jelaskan bagaimana kalian akan menangani permasalahan yang disebutkan pada poin 2
4. Sebutkan dan jelaskan alasan dari teknik encoding yang akan kalian gunakan terhadap data tersebut
5. Buatlah desain eksperimen dengan menentukan hal berikut:
- Tujuan eksperimen
- Dependent dan Independent variabel
- Strategi eksperimen
- Skema validasi
6. Implementasikan strategi eksperimen dan skema validasi yang sudah kalian buat
7. Berdasarkan hasil prediksi yang kalian hasilkan, buatlah kesimpulan analisis karakteristik pasien yang terkena stroke

Poin 1 - 5 dikerjakan saat praktikum berlangsung (pukul 09.00 WIB - 11.00 WIB)
Poin 6 - 7 dikerjakan saat setelah praktikum berlangsung (pukul 11.00 WIB - 23.59 WIB)

Jika terdapat perubahan jawaban pada poin 1 - 5 (semisal perbedaan cara melakukan handling missing value), dapat dijelaskan pada laporan mengenai jawaban sebelum, jawaban sesudah, dan alasan merubah jawaban tersebut (semisal menemukan suatu hal menarik pada data, sehingga missing value dapat dihandle dengan metode yang lebih bagus) 

In [57]:
# 1
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
model_logistic = LogisticRegression(solver='newton-cg')

df = data.copy()
# Melakukan Encoding
categ = ["gender", "ever_married", "smoking_status"]
le = LabelEncoder()
df[categ] = df[categ].apply(le.fit_transform)

oh = OneHotEncoder()
categ = ["work_type", "Residence_type"]
df[categ] = df[categ].apply(le.fit_transform)
# Scaling avg_glucose_level dan bmi
scaler = MinMaxScaler()
scaler.fit(df[["avg_glucose_level"]])
df[["avg_glucose_level"]] = scaler.transform(df[["avg_glucose_level"]])


scaler2 = MinMaxScaler()
df["bmi"].fillna(df["bmi"].mean(), inplace=True)
scaler.fit(df[["bmi"]])
df[["bmi"]] = scaler.transform(df[["bmi"]])
# Fit 
X_log = df.drop(["id","stroke"], axis=1)
y_log = df["stroke"]
X_train, X_test, y_train, y_test = train_test_split(X_log, y_log, test_size=0.2, random_state=123)
model_logistic.fit(X_train,y_train)
# Predict and evaluate
y_pred = model_logistic.predict(X_test)

print(accuracy_score(y_test,y_pred))


0.9510763209393346


In [50]:
# 2

# Analisa duplicate value
feature_col_num = data.select_dtypes(
    include=[np.number]).columns.drop("stroke")
print(data.duplicated().sum())

# Missing Values
print()
feature_col = data.columns.drop("stroke")
for col in feature_col:
    print(f"{col} mempunyai {data[col].isnull().sum()} missing values")
print()


0

id mempunyai 0 missing values
gender mempunyai 0 missing values
age mempunyai 0 missing values
hypertension mempunyai 0 missing values
heart_disease mempunyai 0 missing values
ever_married mempunyai 0 missing values
work_type mempunyai 0 missing values
Residence_type mempunyai 0 missing values
avg_glucose_level mempunyai 0 missing values
bmi mempunyai 201 missing values
smoking_status mempunyai 0 missing values



In [51]:
# Outlier
def detect_outlier(data_1):
    threshold = 3
    mean = np.mean(data_1)
    std = np.std(data_1)
    print(data_1.name)

    for y in data_1.unique():
        z_score = (y - mean)/std
        if np.abs(z_score) > threshold:
            print(f"{y} is an outlier")
    print()


for col in feature_col_num:
    detect_outlier(data[col])


id

age

hypertension
1 is an outlier

heart_disease
1 is an outlier

avg_glucose_level
252.72 is an outlier
243.58 is an outlier
259.63 is an outlier
249.31 is an outlier
263.32 is an outlier
271.74 is an outlier
242.52 is an outlier
250.89 is an outlier
247.51 is an outlier
243.53 is an outlier
242.3 is an outlier
243.5 is an outlier
251.6 is an outlier
247.69 is an outlier
250.2 is an outlier
254.6 is an outlier
254.63 is an outlier
246.34 is an outlier
251.46 is an outlier
267.76 is an outlier
246.53 is an outlier
244.28 is an outlier
251.99 is an outlier
253.16 is an outlier
242.84 is an outlier
249.29 is an outlier
242.94 is an outlier
247.48 is an outlier
266.59 is an outlier
243.73 is an outlier
243.59 is an outlier
250.8 is an outlier
255.17 is an outlier
267.61 is an outlier
260.85 is an outlier
248.37 is an outlier
263.56 is an outlier
247.97 is an outlier
248.24 is an outlier
253.93 is an outlier
254.95 is an outlier
247.87 is an outlier
261.67 is an outlier
256.74 is an ou

In [52]:
#Balance of Data
print()
print(f"Data Training: {Counter(y_train)}")
print(f"Data Validation: {Counter(y_val)}")
print(f"Data Testing: {Counter(y_test)}")


Data Training: Counter({0: 3109, 1: 161})
Data Validation: Counter({0: 780, 1: 38})
Data Testing: Counter({0: 972, 1: 50})


3. Jelaskan bagaimana kalian akan menangani permasalahan yang disebutkan pada poin 2
4. Sebutkan dan jelaskan alasan dari teknik encoding yang akan kalian gunakan terhadap data tersebut
5. Buatlah desain eksperimen dengan menentukan hal berikut:
- Tujuan eksperimen
- Dependent dan Independent variabel
- Strategi eksperimen
- Skema validasi

3.  
- Duplicate Values : Pertama drop kolom stroke, kemudian analisis duplicate values dengan fungsi df.duplicated(), akan me-return boolean values jika row dideteksi memiliki duplicate di tempat lain, lalu kita dapat melakukan penjumlahan terhadap boolean (0 dan 1) tersebut untuk mengetahui jumlah data yang terduplikat. Data duplikat tidak memberikan makna apa-apa ke pembelajaran, sehingga cara terbaik adalah dengan drop instance yang terdeteksi sebagai duplikat(keep 1 dari n-possible duplicate values)  
    
- Missing Values : Pertama drop kolom stroke, kemudian cari missing values dengan fungsi isnull, fungsi isnull() akan me-return boolean values true jika terdeteksi nilai fitur-j pada instance-i bernilai Nan, np.nan, string kosong, dan falsy-values lainnya. Lalu gunakan fungsi sum() untuk menambahkan seluruh boolean values dan mendapatkan jumlah missing values. Dikarenakan tidak terdapat missing values maka tidak perlu dilakukan penanganan khusus untuk dataset ini.
    
- Outlier : Pertama membuat fungsi deteksi outlier, fungsi ini akan melakukan list terhadap seluruh values yang terdeteksi pada outlier pada setiap kolom fitur. Syarat outlier adalah z_score dari value tersebut (selisih absolut value dengan mean dibagi standard deviation) lebih dari 3. Untuk mengatasi outlier ini bisa dilakukan normalisasi dengan MinMaxScaler atau log transformation, sehingga skewness dari dataset tidak condong terlalu ke kanan atau kek kiri.
    
- Balance of Data: Dengan library counter akan menghasilkan perbandingan jumlah dari value kolom target dari data training, data validation, dan data testing. Didapatkan ketidakseimbangan pada data target dalam dataset yang disediakan, dimana untuk data test, training, dan validation kelas 0 memiliki jumlah yang jauh lebih banyak dari kelas 1. Bisa melakukan pembelajaran dengan **undersampling** atau **oversampling** tergantung performa yang lebih baik untuk model yang sama.

4. Untuk gender,dan ever_married dapat dilakukan label encoding, karena hanya memiliki 2 possible values,
Sedangkan untuk Residence_type, job, dan smoking memiliki lebih dari 3 possible values, sehingga jauh lebih baik menggunakan OneHotEncoding karena mesin dapat lebih mengerti perbedaan keluaran dari OneHotEncoding. 

5. 
- Tujuan eksperimen : Mendeteksi seorang pasien apakah memiliki kemungkinan untuk terkena stroke berdasarkan informasi tentang pasien seperti jenis kelamin, umur, penyakit dan status merokok.

- Dependent dan Independent variabel :
    - Dependent :  jenis kelamin, umur, penyakit dan status merokok
    - Independent : stroke
    
- Strategi eksperimen :
    1. Melakukan cleaning terhadap data duplikat, handling missing values,dll.

    2. Melakukan cek terhadap keseimbangan data, ketidakseimbangan memerlukan perlakuan khusus seperti under/over sampling, tergantung yang memberikan performa lebih baik.

    3. Melakukan Encoding terhadap data kategorikal.

    4. Melakukan cek jika terdapat outliers yang cukup signifikan pada kolom numerik, kemungkinannya adalah data mempunyai skewness yang terlalu tajam. Dapat dilakukan scaling dengan teknik MinMaxScaling atau Log Transformation, tergantung hasil terbaik.

- Skema validasi :
Validasi bisa dilakukan dengan menggunakan dataset validation dan dihitung f1 dan accuraccy score. Bisa juga melakukan library cross_val_score yang otomatis membagi dataset awal ke dalam beberapa fold, lalu menghitung score dari model untuk setiap fold.

Jelaskan pembagian tugas/ kerja antar anggota kelompok saat eksperimen

# Deliverables

1. Notebook dengan nama file PraktikumIF3270_M1_NIM1_NIM2.ipynb untuk poin 1 - 5.
2. Notebook dengan nama file PraktikumIF3270_M2_NIM1_NIM2.ipynb yang merupakan kelanjutan dari notebook poin 1, dengan tambahan hasil poin 6 dan 7.
3. Laporan dengan nama file PraktikumIF3270_NIM1_NIM2.pdf dengan isi sebagai berikut:
- Hasil analisa terhadap data, penanganan yang dilakukan serta justifikasi teknik-teknik yang dipilih
- Perubahan yang dilakukan pada jawaban poin 1 - 5 jika ada
- Desain eksperimen
- Hasil eksperimen
- Analisis dan kesimpulan
- Pembagian Tugas / Kerja antar anggota kelompok

Deadline pengumpulan:
- Deliverables poin 1 dikumpulkan sebelum <b>pukul 11.00 WIB</b>, Senin 4 April 2022
- Deliverables poin 2 dikumpulkan sebelum <b>pukul 23.59 WIB</b>, Senin 4 April 2022
- Deliverables poin 3 dikumpulkan sebelum <b>pukul 23.59 WIB</b>, Senin 4 April 2022